In [1]:
!pip install duckdb pandas requests

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 89.6 MB/s eta 0:00:00:00:0100:01


In [36]:
!pip install duckdb


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import requests
import zipfile
import io
import os
import pandas as pd


#Here we can try differnt years but i think the latest is 2024 ia to check and honestlly based on problem statement 
def download_pums_file(state_abbr, record_type, year="2023"):
    """
    record_type: 'p' for person, 'h' for housing
    """
    url = f"https://www2.census.gov/programs-surveys/acs/data/pums/{year}/5-Year/csv_{record_type}{state_abbr}.zip"
    dest_folder = f"data_{state_abbr}"
    os.makedirs(dest_folder, exist_ok=True)
    
    print(f"Downloading {record_type} records for {state_abbr}...")
    response = requests.get(url, stream=True)
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dest_folder)
        print(f"Extracted to {dest_folder}")

# Download  California we only want to do once really 
#download_pums_file("ca", "p")
#download_pums_file("ca", "h")

## Housing Specifically 

In [5]:
clean_housing_cols = [c for c in housing_headers if not c.startswith('WGTP') and not c.startswith('F')]
print(f"Cleaned Housing Variables ({len(clean_housing)}):", clean_housing)


Cleaned Housing Variables (96): ['RT', 'SERIALNO', 'DIVISION', 'PUMA', 'REGION', 'STATE', 'ADJHSG', 'ADJINC', 'NP', 'TYPEHUGQ', 'ACCESSINET', 'ACR', 'AGS', 'BATH', 'BDSP', 'BLD', 'BROADBND', 'COMPOTHX', 'CONP', 'DIALUP', 'ELEFP', 'ELEP', 'GASFP', 'GASP', 'HFL', 'HISPEED', 'HOTWAT', 'INSP', 'LAPTOP', 'MHP', 'MRGI', 'MRGP', 'MRGT', 'MRGX', 'OTHSVCEX', 'REFR', 'RMSP', 'RNTM', 'RNTP', 'RWAT', 'RWATPR', 'SATELLITE', 'SINK', 'SMARTPHONE', 'SMP', 'STOV', 'TABLET', 'TEL', 'TEN', 'VACS', 'VALP', 'VEH', 'WATFP', 'WATP', 'YRBLT', 'CPLT', 'GRNTP', 'GRPIP', 'HHL', 'HHLANP', 'HHLDRAGEP', 'HHLDRHISP', 'HHLDRRAC1P', 'HHT', 'HHT2', 'HINCP', 'HUGCL', 'HUPAC', 'HUPAOC', 'HUPARC', 'KIT', 'LNGI', 'MULTG', 'MV', 'NOC', 'NPF', 'NPP', 'NR', 'NRC', 'OCPIP', 'PARTNER', 'PLM', 'PLMPRP', 'PSF', 'R18', 'R60', 'R65', 'RESMODE', 'SMOCP', 'SMX', 'SRNT', 'SVAL', 'TAXAMT', 'WIF', 'WKEXREL', 'WORKSTAT']


In [13]:
all_headers_housing = pd.read_csv('data_ca/psam_h06.csv', nrows=0).columns.tolist()
print(f"The length of Housing Variables are: {len(all_headers_housing)}")

clean_housing_cols = [c for c in all_headers if not c.startswith('WGTP') and not c.startswith('F')]

The length of Housing Variables are: 237
The length of Person Variables are: 286


In [11]:
import duckdb

# Connect and view the first 5 rows of specific columns
con = duckdb.connect()
sample_query = f"SELECT {', '.join(clean_cols[:10])} FROM 'data_ca/psam_h06.csv' LIMIT 10"
print(con.execute(sample_query).df())

  RT       SERIALNO  DIVISION   PUMA  REGION STATE   ADJHSG   ADJINC  NP  \
0  H  2019GQ0000003         9  03704       4    06  1195583  1207712   1   
1  H  2019GQ0000009         9  07322       4    06  1195583  1207712   1   
2  H  2019GQ0000013         9  05923       4    06  1195583  1207712   1   
3  H  2019GQ0000023         9  07107       4    06  1195583  1207712   1   
4  H  2019GQ0000024         9  08900       4    06  1195583  1207712   1   
5  H  2019GQ0000048         9  03770       4    06  1195583  1207712   1   
6  H  2019GQ0000055         9  11107       4    06  1195583  1207712   1   
7  H  2019GQ0000064         9  03703       4    06  1195583  1207712   1   
8  H  2019GQ0000070         9  11104       4    06  1195583  1207712   1   
9  H  2019GQ0000074         9  02906       4    06  1195583  1207712   1   

   TYPEHUGQ  
0         3  
1         2  
2         3  
3         2  
4         3  
5         2  
6         3  
7         2  
8         3  
9         2  


In [14]:
full_row_check = con.execute("SELECT * FROM 'data_ca/psam_h06.csv' LIMIT 1").df().T

print(full_row_check)

                      0
RT                    H
SERIALNO  2019GQ0000003
DIVISION              9
PUMA              03704
REGION                4
...                 ...
WGTP76                0
WGTP77                0
WGTP78                0
WGTP79                0
WGTP80                0

[237 rows x 1 columns]


## Persons Specifically 

In [16]:



all_headers_persons = pd.read_csv('data_ca/psam_p06.csv', nrows=0).columns.tolist()
print(f"The length of Person Variables are: {len(all_headers_persons)}")
clean_persons_cols = [c for c in all_headers_persons if not c.startswith('WGTP') and not c.startswith('F')]


The length of Person Variables are: 286


In [15]:
!du -h data_ca/ | sort -h

257K	data_ca/.ipynb_checkpoints
788M	data_ca/


### I guess we can think about the actual architecture or ways of manipulating the file 

import duckdb

#  connection to a local DuckDB file (this acts as  storage)
con = duckdb.connect('acs_california_2023.db')

#  Register the CSVs as virtual tablesm ??? (without loading them into RAM)

con.execute("""
    CREATE TABLE person_raw AS SELECT * FROM read_csv_auto('data_ca/psam_p06.csv');
    CREATE TABLE housing_raw AS SELECT * FROM read_csv_auto('data_ca/psam_h06.csv');
""")

# 2. Join  and select only the columns needed, here we must figure out what we acually want to use ... to keep table small
con.execute("""
    CREATE TABLE ml_ready_data AS 
    SELECT 
        p.SERIALNO, 
        p.AGEP, 
        p.SEX, 
        p.SCHL, 
        p.ESR, -- Employment status
        h.HINCP, -- Household Income
        h.NP,    -- Number of persons in household
        h.VEH    -- Number of vehicles
    FROM person_raw p
    JOIN housing_raw h ON p.SERIALNO = h.SERIALNO
""")

print(con.execute("SELECT count(*) FROM ml_ready_data").fetchall())